In [1]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
import glob
import datetime
import traceback

## MongoDB

In [2]:
client = MongoClient("mongodb://localhost:27017/")
db = client["local"]
presidencial = db["mesas-presidencial"]
distrital = db["mesas-distrital"]
dip_nacional = db["mesas-nacional"]
municipal = db["mesas-alcaldias"]

In [15]:
def limpiar_mongo(collections):
    for col in collections:
        print(f'eliminados {col.delete_many({}).deleted_count} documentos de {col.name}')

In [16]:
collections = [presidencial, dip_nacional, distrital, municipal]
limpiar_mongo(collections)

eliminados 24296 documentos de mesas-presidencial
eliminados 24103 documentos de mesas-nacional
eliminados 24122 documentos de mesas-distrital
eliminados 24145 documentos de mesas-alcaldias


## Leer Datos TSE

In [3]:
def leer_csv_tse(path):
    df = pd.read_csv(path,  sep=',', header=4, index_col=0)
    df = df.drop(columns=['OBSERVACIONES', 'CÓDIGO_INTEGRIDAD', 'FECHA_HORA_PUBLICACIÓN', 'DIGITALIZACIÓN', 'ID_DEPARTAMENTO', 'ID_MUNICIPIO', 'PADRÓN'])
    df['ARCHIVO'] = path
    df['FECHA_ARCHIVO'] = pd.read_csv(path, nrows=1, skiprows=[0], header=None).iloc[0,0]
    return df

def ingerir_directorio_tse(path, collection):
    csv_paths = glob.glob(path + r'\*.csv')
    result = []
    for csv_path in csv_paths:
        no_contabilizadas = pd.NA
        insertadas = 0
        try:
            data = leer_csv_tse(csv_path)
            no_contabilizadas = cuenta_no_contabilizadas(data)
            data = solo_contabilizadas(data)
            if data.empty:
                print(f'archivo no contiene actas contabilizadas {csv_path}')
            else:    
                insert_result = collection.insert_many(convertir_a_mongo_docs(data))
                insertadas = len(insert_result.inserted_ids)
        except Exception as e:
                print(f"archivo: {csv_path} error: {str(e)}")
        file_result = {'archivo' : csv_path, ' no_contabilizadas' : no_contabilizadas, 'insertadas' : insertadas}
        result.append(file_result)
    return pd.DataFrame(result)

def llenar_nans(data):
    return data.applymap(lambda l: l if not pd.isna(l) else np.random.choice(100))

def solo_contabilizadas(df):
    return df[df['CONTABILIZADA'] == 1]

def cuenta_no_contabilizadas(df):
    return len(df[df['CONTABILIZADA'] != 1])

def convertir_a_mongo_docs(data):
    data_dict = data.transpose().to_dict()
    for mesa in data_dict:
        data_dict[mesa]['mesa'] = mesa
    return list(data_dict.values())

## Leer Datos Fiscales

In [4]:
def leer_excel_fiscales(path):
    xlsx = pd.ExcelFile(path)
    hojas_mesas = xlsx.sheet_names[1:-1]
    xlsx = pd.read_excel(path, header=5, index_col=1, sheet_name=hojas_mesas)
    dataframes = {}
    for hoja in xlsx:
        #print(path, hoja)
        df = xlsx[hoja]
        if df.empty:
            continue
        df = df.drop(df.columns[0], axis=1).transpose()
        #df.columns = df.columns.str.strip()
        nan_header_column_index = df.columns.tolist().index(np.nan)
        df = df.iloc[:, :nan_header_column_index]
        if not df.empty:
            dataframes[hoja]=df
    return dataframes    
    
#list(leer_excel_fiscales('excel-pruebas\Pruebas-AV.xlsx').values())[0]

In [ ]:
def nombres_columnas(directorio):
    xlsx_paths = glob.glob(directorio + r'\**\*.xlsx', recursive=True)
    result = set()
    
    for path in xlsx_paths:
        try:
            hojas = leer_excel_fiscales(path)
            for hoja in hojas.keys():
                nombres = hojas[hoja].columns.to_list()
                result.update(nombres)
        except Exception as e:
            print(path, e)
            traceback.print_exc()
            continue
   
    return sorted(result)


print(datetime.datetime.now().time())
nombres = nombres_columnas(r'C:\Users\batoz\Downloads\Elecciones 2023-20230629-00')
print(datetime.datetime.now().time())
nombres

## Comparar

In [84]:
def find_mesas(mesa_inicio, mesa_fin, collection):
    query = {"mesa": {"$gte": mesa_inicio, "$lte": mesa_fin}}    
    result = {}
    for datos_mesa in collection.find(query):
        result[datos_mesa['mesa']] = datos_mesa
    return result

def to_int_or_nan(value, key):
    vaue = value.strip() if isinstance(value, str) else value
    if not pd.isna(value) and isinstance(value, str) and not value.isnumeric():
        if value != '--':
            print(f'WARN - found value {value} for key {key}')
        return 0 if value == '--' else np.NaN
    return int(value) if not pd.isna(value) else np.NaN

def comparar_mesa(no_mesa, datos_mesa_fiscal, datos_mesa_tse, path):
    result = []
    mismatched_cols = []
    keys = list(datos_mesa_fiscal.keys())
    for key in keys:
        if key in diccionario_partidos:
            new_key = diccionario_partidos[key]
            datos_mesa_fiscal[new_key] = datos_mesa_fiscal.pop(key)
            key = new_key
        if key in datos_mesa_tse:
            valor_fiscal = to_int_or_nan(datos_mesa_fiscal[key], key)
            valor_tse = to_int_or_nan(datos_mesa_tse[key], key)
            if not np.isnan(valor_fiscal) and valor_fiscal != valor_tse:
                result.append({
                    'mesa' : no_mesa,
                    'columna' : key,
                    'tse': valor_tse,
                    'fiscal': valor_fiscal,
                    'dif' :np.nan_to_num(np.subtract(valor_tse, valor_fiscal))
                })
        else:
            mismatched_cols.append(key)
    return (result, mismatched_cols)
            

def comparar_archivo(path, mongo_collection):
    result = []
    mismatched_cols = {}
    hojas = leer_excel_fiscales(path)
    for hoja in hojas.keys():
        mesas_excel = hojas[hoja]
        mesa_inicio = mesas_excel.iloc[0].name
        mesa_fin = mesas_excel.iloc[-1].name
        print(mesa_inicio, mesa_fin, path)
        mesas_db = find_mesas(mesa_inicio, mesa_fin, mongo_collection)
        for no_mesa, mesa_fiscal in mesas_excel.iterrows():
            if no_mesa in mesas_db:
                mesa_fiscal = mesa_fiscal.to_dict()
                mesa_tse = mesas_db[no_mesa]
                (diferencias, mismatches) = comparar_mesa(no_mesa, mesa_fiscal, mesa_tse, path)
                result.extend(diferencias)
                mismatched_cols[hoja] = mismatches
    return (pd.DataFrame(result), mismatched_cols)

pd.set_option('display.max_rows', 1000)
#comparar_archivo('excel-pruebas\Chimaltenango.xlsx', distrital)[0][lambda x: x['columna'] == 'MLP']


#hojas = leer_excel_fiscales('excel-pruebas\Chimaltenango.xlsx')['EL TEJAR']
#mesa_tse = hojas.loc[6794]
#mesa_db = find_mesas(6794, 6794, distrital)
#comparar_mesa(6794, mesa_db, mesa_tse, '')

In [81]:
def comparar_todos_los_archivos(directorio, collection):
    xlsx_paths = glob.glob(directorio + r'\*.xlsx')
    result = pd.DataFrame()
    mismatches = {}
    for path in xlsx_paths:
        (diffs, col_mismatches) = comparar_archivo(path, collection)
        result = pd.concat([result, diffs])
        mismatches[path] = col_mismatches
    return (result, mismatches)




fisc = leer_excel_fiscales(r'C:\Users\batoz\Documents\elecciones\datos-fiscales\manuales\presidencial\Sacatepequez.xlsx')['ANTIGUA GUATEMALA'].loc[5338]

tse = find_mesas(5338, 5338, presidencial)[5338]

#comparar_mesa(5338, fisc.to_dict(), tse, 'a')[0][0]


(result, mismatches) = comparar_todos_los_archivos(r'datos-fiscales\manuales\presidencial', presidencial)
(mismatches, result)
#comparar_archivo(r'datos-fiscales\manuales\presidencial\Sacatepequez.xlsx', presidencial)

5337 5436
5636 5680
5708 5726


({'datos-fiscales\\manuales\\presidencial\\Sacatepequez.xlsx': {'ANTIGUA GUATEMALA': [],
   'SAN LUCAS SACATEPÉQUEZ': ['U R N G - MAIZ'],
   'MAGDALENA MILPAS ALTAS': []}},
     mesa          columna  tse  fiscal  dif
 0   5338              UNE  111      11  100
 1   5339           BLANCO    1      96  -95
 2   5342          SEMILLA   91       1   90
 3   5344          VÁLIDOS  275     304  -29
 4   5344            TOTAL  304     333  -29
 5   5350            TOTAL  292     283    9
 6   5353              UNE   12       4    8
 7   5353             AZUL    1       2   -1
 8   5353  VALOR UNIONISTA   32       3   29
 9   5353            CABAL   25      18    7
 10  5353            TODOS    2      14  -12
 11  5353            VAMOS    5       3    2
 12  5353              PHG    1      10   -9
 13  5353               PR    6      11   -5
 14  5353              PIN    0       5   -5
 15  5353         ELEFANTE    3       4   -1
 16  5353         VICTORIA    1       2   -1
 17  5353        

In [85]:
(result, mismatches) = comparar_todos_los_archivos(r'C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Distrital', distrital)
result[lambda x: x['columna'] == 'SEMILLA']

18909 18955 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Distrital\AltaVerapaz.xlsx
19053 19107 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Distrital\AltaVerapaz.xlsx
19432 19739 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Distrital\AltaVerapaz.xlsx
19982 20098 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Distrital\AltaVerapaz.xlsx
18355 18447 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Distrital\BajaVerapaz.xlsx
18497 18535 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Distrital\BajaVerapaz.xlsx
6608 6658 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Distrital\Chimaltenango.xlsx
6731 6768 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Distrital\Chimaltenango.xlsx
6769 6809 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Distrital\Chimaltenango.xlsx
1 2023 C:\Users\batoz\Downlo

,mesa,columna,tse,fiscal,dif
16,19516,SEMILLA,0,4,-4
19,6768,SEMILLA,5,22,-17
45,6785,SEMILLA,14,31,-17
58,6806,SEMILLA,46,6,40
60,6807,SEMILLA,21,1,20
20,51,SEMILLA,63,44,19
27,1166,SEMILLA,85,90,-5
21,23072,SEMILLA,0,24,-24
46,23151,SEMILLA,11,2,9
94,23228,SEMILLA,5,1,4


In [89]:
(result, mismatches) = comparar_todos_los_archivos(r'C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Listado Nacional', dip_nacional)
result.sort_values('dif')

18613 18908 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Listado Nacional\AltaVerapaz.xlsx
18909 18955 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Listado Nacional\AltaVerapaz.xlsx
19053 19107 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Listado Nacional\AltaVerapaz.xlsx
19432 19739 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Listado Nacional\AltaVerapaz.xlsx
19982 20098 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Listado Nacional\AltaVerapaz.xlsx
18355 18447 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Listado Nacional\BajaVerapaz.xlsx
18497 18535 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Listado Nacional\BajaVerapaz.xlsx
18536 18612 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Listado Nacional\BajaVerapaz.xlsx
6392 6482 C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\L

,mesa,columna,tse,fiscal,dif
36,1147,BLANCO,0,400,-400
48,5678,TOTAL,0,312,-312
60,17815,TOTAL,0,286,-286
2,2950,TOTAL,0,278,-278
64,17834,TOTAL,0,264,-264
...,...,...,...,...,...
26,6774,TOTAL,284,0,284
28,6776,TOTAL,288,0,288
27,6775,TOTAL,291,0,291
25,6772,TOTAL,292,0,292


In [99]:
pd.set_option('display.max_rows', 2000)

result.sort_values('dif')[lambda x: x['columna'] != 'TOTAL'][lambda x: x['columna'] != 'BLANCO'][lambda x: x['columna'] != 'VÁLIDOS'][lambda x: x['columna'] != 'NULOS']   


,mesa,columna,tse,fiscal,dif
600,23160,SEMILLA,0,244,-244
574,23159,SEMILLA,2,238,-236
395,23110,SEMILLA,34,269,-235
179,23093,SEMILLA,24,256,-232
125,23088,SEMILLA,24,252,-228
543,23155,SEMILLA,2,228,-226
223,23097,SEMILLA,8,232,-224
451,23124,SEMILLA,16,237,-221
424,23122,SEMILLA,26,246,-220
45,23071,SEMILLA,41,259,-218


## Llenar base de datos

In [18]:
limpiar_mongo([presidencial])
ingerir_directorio_tse(r'datos-tse\reales\1-PRESIDENTE', presidencial)

eliminados 24296 documentos de mesas-presidencial
eliminados 24103 documentos de mesas-nacional
eliminados 24122 documentos de mesas-distrital
eliminados 24145 documentos de mesas-alcaldias


eliminados 24296 documentos de mesas-presidencial


,archivo,no_contabilizadas,insertadas
0,datos-tse\reales\1-PRESIDENTE\gtm2023_e1.csv,289,24296


In [20]:
limpiar_mongo([dip_nacional])
ingerir_directorio_tse(r'datos-tse\reales\2-DIP-NAC', dip_nacional)

eliminados 24104 documentos de mesas-nacional


,archivo,no_contabilizadas,insertadas
0,datos-tse\reales\2-DIP-NAC\gtm2023_e2.csv,323,24104


In [22]:
limpiar_mongo([distrital])
ingerir_directorio_tse(r'datos-tse\reales\3-DIP-DIST', distrital)

eliminados 24123 documentos de mesas-distrital


,archivo,no_contabilizadas,insertadas
0,datos-tse\reales\3-DIP-DIST\gtm2023_e3d00.csv,13,2010
1,datos-tse\reales\3-DIP-DIST\gtm2023_e3d01.csv,71,3242
2,datos-tse\reales\3-DIP-DIST\gtm2023_e3d02.csv,1,550
3,datos-tse\reales\3-DIP-DIST\gtm2023_e3d03.csv,48,874
4,datos-tse\reales\3-DIP-DIST\gtm2023_e3d04.csv,0,346
5,datos-tse\reales\3-DIP-DIST\gtm2023_e3d05.csv,14,1164
6,datos-tse\reales\3-DIP-DIST\gtm2023_e3d06.csv,5,704
7,datos-tse\reales\3-DIP-DIST\gtm2023_e3d07.csv,2,704
8,datos-tse\reales\3-DIP-DIST\gtm2023_e3d08.csv,2,656
9,datos-tse\reales\3-DIP-DIST\gtm2023_e3d09.csv,2,1318


In [24]:
limpiar_mongo([municipal])
ingerir_directorio_tse(r'datos-tse\reales\4-CORPORACIÓN', municipal)

eliminados 24146 documentos de mesas-alcaldias
archivo no contiene actas contabilizadas datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m04.csv
archivo no contiene actas contabilizadas datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d11m04.csv


,archivo,no_contabilizadas,insertadas
0,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d00m0...,8,2015
1,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,0,151
2,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,0,122
3,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,24,0
4,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,0,102
5,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,32,178
6,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,0,90
7,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,3,742
8,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,0,73
9,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m1...,1,323


## Diccionario

In [41]:
diccionario_partidos = {
    ' BLA NCO': 'BLANCO',
    'BLANCO': 'BLANCO',
    'BLANCOS': 'BLANCO',
    'Blanco': 'BLANCO',
    'EN BLANCO': 'BLANCO',
    'EN BLANCO ': 'BLANCO',
    'ENBLANCO': 'BLANCO',
    'En Blanco': 'BLANCO',
    'En blanco': 'BLANCO',
    'HOJAS EN BLANCO':  'BLANCO',
    'HOJAS EN BLANCO ': 'BLANCO',
    'VOTOS BLANCO': 'BLANCO',
    'VOTOS EN BLANCO':  'BLANCO',
    'VOTOS EN BLANCO ': 'BLANCO',
    'VOTOS EN BLANCOS': 'BLANCO',
    'Votos en Blanco': 'BLANCO',
    'Votos en blanco': 'BLANCO',
    'en blanco': 'BLANCO',
    'votos blancos': 'BLANCO',
    'votos en blanco': 'BLANCO',
    'votos en blanco ': 'BLANCO',

    ' NULOS': 'NULOS',
    ' nulos': 'NULOS',
    'NULO': 'NULOS',
    'NULOS': 'NULOS',
    'NULOS ': 'Nulos ',
    'Nulos': 'NULOS',
    'VOTOS NULOS': 'NULOS',
    'VOTOS NULOS ': 'NULOS',
    'Voto nulo': 'NULOS',
    'Votos Nulos': 'NULOS',
    'nulos': 'NULOS',
    'votos nulos': 'NULOS',

    ' VALIDOS ': 'VÁLIDOS',
    'TOTAL DE VÁLIDOS ': 'VÁLIDOS',
    'TOTAL VALIDOS': 'VÁLIDOS',
    'TOTAL VOTOS VALIDAMENTE EMITIDOS': 'VÁLIDOS',
    'TOTAL VOTOS VALIDANTE EMITIDOS': 'VÁLIDOS',
    'TOTAL VOTOS VALIDOS': 'VÁLIDOS',
    'TOTAL VOTOS VALIDOS EMITIDOS': 'VÁLIDOS',
    'TOTAL VOTOS VPALIDOS': 'VÁLIDOS',
    'TOTAL VOTOS VÁLIDOS': 'VÁLIDOS',
    'TOTAL VOTOS VÁLIDOS ': 'VÁLIDOS',
    'Válidos': 'VÁLIDOS',
    'Total Votos Validos': 'VÁLIDOS',
    'VALIDOS': 'VÁLIDOS',
    'VALIDOS ': 'VÁLIDOS',
    'VOTOS VALIDOS': 'VÁLIDOS',
    'VOTOS VALIDOS ': 'VÁLIDOS',
    'Validos': 'VÁLIDOS',
    'total votos validos': 'VÁLIDOS',
    'validos': 'VÁLIDOS',
    'votos validos': 'VÁLIDOS',
    ' TOTAL DE VOTOS VALIDAMENTE EMITIDOS': 'VÁLIDOS',
    'total de votos válidamente emitidos ': 'VÁLIDOS',
    'total votos': 'VÁLIDOS',
    'total votos ': 'VÁLIDOS',
    'VOTOS VALIDAMENTE EMITIDOS': 'VÁLIDOS',
    'Votos valido': 'VÁLIDOS',
    'total votos validamente emitidos': 'VÁLIDOS',
    'TOTAL DE VOTOS VALIDAMENTE EMITIDOS': 'VÁLIDOS',
    'TOTAL DE VOTOS VALIDAMENTE EMITIDOS ': 'VÁLIDOS',
    'Total de Votos Válidamente Emitidos': 'VÁLIDOS',
    'Total de votos válidamente emitidos ': 'VÁLIDOS',
    'VOTOS EMITIDOS': 'VÁLIDOS',
    'VOTOS EMITIDOS ': 'VÁLIDOS ',

    'INVÁLIDOS ': 'INVÁLIDOS',
    'Total votos inválidos': 'INVÁLIDOS',
    'VOTOS INVÁLIDOS': 'INVÁLIDOS',
    'votos inválidos ': 'INVÁLIDOS',
    'INVALIDOS': 'INVÁLIDOS',
    'INVALIDOS ': 'INVÁLIDOS',
    'TOTO INVALIDOS': 'INVÁLIDOS',
    'VOTOS INVALIDOS': 'INVÁLIDOS',
    'VOTOS INVALIDOS ': 'INVÁLIDOS',
    'Votos invalidos ': 'INVÁLIDOS',
    'votos invalidos': 'INVÁLIDOS',

    ' PARTIDO AZUL (AZUL)': 'AZUL',
    'AZUL': 'AZUL',
    'AZUL ': 'AZUL',
    'Azul': 'AZUL',
    'PARTIDO (AZUL)': 'AZUL',
    'PARTIDO AZU (AZUL)': 'AZUL',
    'PARTIDO AZUL': 'AZUL',
    'PARTIDO AZUL ': 'AZUL',
    'PARTIDO AZUL (AZUL )': 'AZUL',
    'PARTIDO AZUL (AZUL)': 'AZUL',
    'PARTIDO AZUL(AZUL)': 'AZUL',
    'Partido Azul': 'AZUL',
    'Partido Azul (Azul)': 'AZUL',
    'azul': 'AZUL',
    'azul ': 'AZUL',
    'partido azul': 'AZUL',

    ' vos': 'VOS',
    'BOLUNTAD , OPORTUNIDAD Y SOLIRIDAD (VOS)': 'VOS',
    'PARTIDO VOS': 'VOS',
    'VOLUNTAD OPORTUNIDAD SOLIDARIDAD (VOS)': 'VOS',
    'VOLUNTAD OPORTUNIDAD Y  SOLIDARIDAD (VOS)': 'VOS',
    'VOLUNTAD OPORTUNIDAD Y SOLIDARIDAD (VOS)': 'VOS',
    'VOLUNTAD OPORTUNIDAD Y SOLIDARIDAD (VOS) ': 'VOS',
    'VOLUNTAD OPORTUNIDAD Y SOLIDARIDAD(VOS)': 'VOS',
    'VOLUNTAD OPORTUNUDAD Y SOLIDARIDAD  (VOS)': 'VOS',
    'VOLUNTAD, OPORTUNDAD Y SOLIDARIDAD(VOS)': 'VOS',
    'VOLUNTAD, OPORTUNIDAD Y SOLIDARIDAD  (VOS)': 'VOS',
    'VOLUNTAD, OPORTUNIDAD Y SOLIDARIDAD (VOS': 'VOS',
    'VOLUNTAD, OPORTUNIDAD Y SOLIDARIDAD (VOS)': 'VOS',
    'VOLUNTAD, OPORTUNIDAD Y SOLIDARIDAD (VOS) ': 'VOS',
    'VOLUNTAD, OPORTUNIDAD Y SOLIDARIDAD(VOS)': 'VOS',
    'VOLUNTAD, OPORTUNIDAD Y SOLIDARIDAS (VOS)': 'VOS',
    'VOLUNTAD,OPORTUNIDAD Y SOLIDARIDAD (VOS)': 'VOS',
    'VOLUNTAD,OPORTUNIDAD, SOLIDARIDAD (VOS)': 'VOS',
    'VOS': 'VOS',
    'VOS ': 'VOS',
    'Volunta Oportunidad y Solidaridad (VOS)': 'VOS',
    'Voluntad oportunidad y solidaridad (VOS)': 'VOS',
    'Voluntad, Oportunidad y Solidaridad (Vos)': 'VOS',
    'Vos': 'VOS',
    'partido vos': 'VOS',
    'vos': 'VOS',
    'VOLUNTAD, OPORTUNIDAD Y SOLIDARIDAD': 'VOS',
    'Voluntad Oportunidad y Solidaridad': 'VOS',
    'voluntad,oportunidad y solidaridad': 'VOS',
    'VOLINTAD, OPORTUNUDAD Y SOLIDARIDAD': 'VOS',
    'VOZ': 'VOS',

    '(Bien': 'BIEN',
    'BIEN': 'BIEN',
    'BIEN ': 'BIEN',
    'BIENESTAR': 'BIEN',
    'BIENESTAR NACIONA (BIEN)': 'BIEN',
    'BIENESTAR NACIONAL': 'BIEN',
    'BIENESTAR NACIONAL ': 'BIEN',
    'BIENESTAR NACIONAL (BIEN ': 'BIEN',
    'BIENESTAR NACIONAL (BIEN)': 'BIEN',
    'BIENESTAR NACIONAL (BIM)': 'BIEN',
    'BIENESTAR NACIONAL(BIEN)': 'BIEN',
    'BIENESTR NACIONAL  ': 'BIEN',
    'Bien': 'BIEN',
    'Bienestar Nacional (BIEN)': 'BIEN',
    'Bienestar Nacional (Bien)': 'BIEN',
    'Bienestar Nacional Bien': 'BIEN',
    'Bienestar nacional': 'BIEN',
    'VIENESTAR NACIONAL (BIEN)': 'BIEN',
    'bien': 'BIEN',

    '(une)': '(une)',
    'PARTIDO UNE': 'PARTIDO UNE',
    'UNE': 'UNE',
    'UNE ': 'UNE ',
    'UNIDAD DE LA ESPERANZA (UNE)': 'UNE',
    'UNIDAD NACIONA DE LA ESPERANZA (UNE)': 'UNE',
    'UNIDAD NACIONAL DE ESPERANZA (UNE)': 'UNE',
    'UNIDAD NACIONAL DE LA ESPEFRANZA (UNE)': 'UNE',
    'UNIDAD NACIONAL DE LA ESPERANSA  (UNE)': 'UNE',
    'UNIDAD NACIONAL DE LA ESPERANZA  (UNE)': 'UNE',
    'UNIDAD NACIONAL DE LA ESPERANZA (UNE )': 'UNE',
    'UNIDAD NACIONAL DE LA ESPERANZA (UNE)': 'UNE',
    'UNIDAD NACIONAL DE LA ESPERZANZA  (UNE)': 'UNE',
    'UNIDAD NACIONAL DE LA ES´PERANZA (UNE)': 'UNE',
    'UNIDAD NACIONAL DE LA EZPERANZA (UNE)': 'UNE',
    'Une': 'UNE',
    'Unidad Nacional de la Esperanza (UNE)': 'UNE',
    'une': 'UNE',
    'PARTIDO NACIONAL DE LA ESPERANZA': 'UNE',
    'UNIDAD LA ESPERANZA': 'UNE',
    'UNIDAD NACIONAL DE LA ESPERANZA': 'UNE',
    'unidad nacional  de la esperanza': 'UNE',
    'unidad nacional de la esperanza': 'UNE',

    '(vamos)': 'VAMOS',
    'PARTIDO VAMOS': 'VAMOS',
    'VAMOS': 'VAMOS',
    'VAMOS ': 'VAMOS',
    'VAMOS POR UN GUATEMALA DIFERENTE (VAMOS)': 'VAMOS',
    'VAMOS POR UNA GUATEMALA  DIFERENTE (VAMOS)': 'VAMOS',
    'VAMOS POR UNA GUATEMALA DEFERENTE': 'VAMOS',
    'VAMOS POR UNA GUATEMALA DIFERENTA (VAMOS)': 'VAMOS',
    'VAMOS POR UNA GUATEMALA DIFERENTE': 'VAMOS',
    'VAMOS POR UNA GUATEMALA DIFERENTE (VAMOS )': 'VAMOS',
    'VAMOS POR UNA GUATEMALA DIFERENTE (VAMOS)': 'VAMOS',
    'Vamos': 'VAMOS',
    'Vamos por una Guatemala Diferente (Vamos)': 'VAMOS',
    'Vamos por una Guatemala diferente (Vamos)': 'VAMOS',
    'vamos': 'VAMOS',
    'vamos por una Guatemala diferente ': 'VAMOS',
    'vamos por una guatemala diferente': 'VAMOS',

    '(victoria': 'VICTORIA',
    'PARTIDO VICTORIA': 'VICTORIA',
    'VICTORIA': 'VICTORIA',
    'VICTORIA ': 'VICTORIA',
    'VICTORIA  (VICTORIA)': 'VICTORIA',
    'VICTORIA (VICTORIA )': 'VICTORIA',
    'VICTORIA (VICTORIA)': 'VICTORIA',
    'VICTORIA (VICTORIA) ': 'VICTORIA',
    'Victoria': 'VICTORIA',
    'Victoria ': 'VICTORIA',
    'victoria': 'VICTORIA',
    'VICTORIIA': 'VICTORIA',

    'BICION CON VALORES (VIVA)': 'VIVA',
    'PARTIDO VIVA': 'VIVA',
    'VICION CON VALORES (VIVA)': 'VIVA',
    'VISION CON LALORES (VIVA)': 'VIVA',
    'VISION CON VALORES (VIVA)': 'VIVA',
    'VISION CON VALORES (VIVA) ': 'VIVA',
    'VISION CON VALORES VIVA': 'VIVA',
    'VISION CON VALORES(VIVA)': 'VIVA',
    'VISIÓN CON VALORES (VIVA)': 'VIVA',
    'VIVA': 'VIVA',
    'VIVA ': 'VIVA',
    'Visión Con Valores VIVA': 'VIVA',
    'Visión con Valores (VIVA)': 'VIVA',
    'Viva': 'VIVA',
    'partido viva': 'VIVA',
    'viva': 'VIVA',
    'VISION CON VALORES': 'VIVA',
    'Vision Con Valores': 'VIVA',


    'CABAL': 'CABAL',
    'CABAL ': 'CABAL',
    'CABAL  (CABAL)': 'CABAL',
    'CABAL (CABAL': 'CABAL',
    'CABAL (CABAL )': 'CABAL',
    'CABAL (CABAL)': 'CABAL',
    'CABAL (CABAL) ': 'CABAL',
    'CABAL(CABAL)': 'CABAL',
    'Cabal': 'CABAL',
    'PARTIDO CABAL': 'CABAL',
    'cabal': 'CABAL',
    'CAVAL(CAVAL)': 'CABAL',

    'CAMBIO': 'CAMBIO',
    'CAMBIO ': 'CAMBIO',
    'CAMBIO  (CAMBIO)': 'CAMBIO',
    'CAMBIO (CAMBIO)': 'CAMBIO',
    'CAMBIO(CAMBIO)': 'CAMBIO',
    'Cambio': 'CAMBIO',
    'PARTIDO CAMBIO': 'CAMBIO',
    'cambio': 'CAMBIO',
    'CAMIO': 'CAMBIO',
    'cambió ': 'CAMBIO',

    'COMPROMISO REBOLUCIONARIO Y ORDEN (CREO)': 'CREO',
    'COMPROMISO RENOVACION ORDEN (CREO)': 'CREO',
    'COMPROMISO RENOVACION Y ORDEN (CREO)': 'CREO',
    'COMPROMISO RENOVACIÓN Y ORDEN (CREO)': 'CREO',
    'COMPROMISO, RENOBACION Y ORDEN (CREO)': 'CREO',
    'COMPROMISO, RENOVACION Y ORDEN  (CREO)': 'CREO',
    'COMPROMISO, RENOVACION Y ORDEN (CREO)': 'CREO',
    'COMPROMISO, RENOVACION Y ORDEN(CREO)': 'CREO',
    'COMPROMISO, RENOVACIÒN Y ORDEN (CREO)': 'CREO',
    'COMPROMISO, RENOVACIÓN Y ORDEN (CREO)': 'CREO',
    'COMPROMISO, RENOVACIÓN Y ORDEN(CREO)': 'CREO',
    'COPROMISO RENOVACION Y ORDEN (CREO)': 'CREO',
    'CREO': 'CREO',
    'CREO ': 'CREO',
    'CREO-UNIONISTA': 'CREO',
    'Compromiso Renovación y Orden (CREO)': 'CREO',
    'Compromiso, Renovación y Orden (CREO)': 'CREO',
    'Creo': 'CREO',
    'creo': 'CREO',
    'partido creo': 'CREO',
    'COMPROMISO, RENOVACION Y ORDEN': 'CREO',
    'compromiso renovacion y orden': 'CREO',

    'COMUNIDAD EL ELEFANTE': 'ELEFANTE',
    'COMUNIDAD ELEFANTE': 'ELEFANTE',
    'COMUNIDAD ELEFANTE ': 'ELEFANTE',
    'COMUNIDAD ELEFANTE  (ELEFANTE)': 'ELEFANTE',
    'COMUNIDAD ELEFANTE (ELEFANTE )': 'ELEFANTE',
    'COMUNIDAD ELEFANTE (ELEFANTE)': 'ELEFANTE',
    'COMUNIDAD ELEFANTE (ELEFANTE) ': 'ELEFANTE',
    'COMUNIDAD ELEFANTE(ELEFANTE)': 'ELEFANTE',
    'Comunidad Elefante': 'ELEFANTE',
    'Comunidad Elefante (Elefante)': 'ELEFANTE',
    'ELEFANTE': 'ELEFANTE',
    'ELEFANTE ': 'ELEFANTE',
    'Elefante': 'ELEFANTE',
    'Elefante ': 'ELEFANTE',
    'PARTIDO ELEFANTE ': 'ELEFANTE',
    'comunidad elefante': 'ELEFANTE',
    'elefante': 'ELEFANTE',

    'FCN': 'FCN-NACION',
    'FCN NACION': 'FCN-NACION',
    'FCN NACION ': 'FCN-NACION',
    'FCN NACIONAL': 'FCN-NACION',
    'FCN Nacion': 'FCN-NACION',
    'FCN NACIÓN': 'FCN-NACION',
    'FCN Nación': 'FCN-NACION',
    'FCN nación ': 'FCN-NACION',
    'FCN- NACION': 'FCN-NACION',
    'FCN-NACION': 'FCN-NACION',
    'FCN-NACION ': 'FCN-NACION',
    'FCNacion': 'FCN-NACION',
    'FRENDE DE CONVERGENCIA NACIONAL (FCN-NACION)': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL (FCN NACION)': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL (FCN NACIÓN)': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL (FCN-NACION ': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL (FCN-NACION)': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL (FCN-NACIONAL)': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL (FCN-NACIÓN)': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL (FCN. NACION)': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL(FCN NACION)': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL(FCN-NACION)': 'FCN-NACION',
    'FRENTE DE RENOVACION Y ORDEN (FCN NACION)': 'FCN-NACION',
    'Frente de Convergencia Nacional (FCN)': 'FCN-NACION',
    'Frente de Convergencia Nacional FCN': 'FCN-NACION',
    'fcn': 'FCN-NACION',
    'F.C.N': 'FCN-NACION',
    'F.C.N.': 'FCN-NACION',
    'FNC': 'FCN-NACION',
    'FNC-NACION': 'FCN-NACION',
    'FNNacion': 'FCN-NACION',
    'FRENTE DE CONBERGENCIA NACIONAL ': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL (FCM-NACION)': 'FCN-NACION',
    'FRENTE DE CONVERGENCIA NACIONAL (FNC-NACION)': 'FCN-NACION',
    'Frente de Convergencia Nacional': 'FCN-NACION',
    'frente de convergencia nacional': 'FCN-NACION',

    'HUMANISTA': 'PHG',
    'HUMANISTA ': 'PHG',
    'HUMANISTA (PNG)': 'PHG',
    'HUMANISTA DE GUATEMALA': 'PHG',
    'HUMANISTA DE GUATEMALA P.H.G': 'PHG',
    'HUMANISTA PHG': 'PHG',
    'Humanista': 'PHG',
    'Humanista ': 'PHG',
    'PARTIDO HUMANISTA': 'PHG',
    'PARTIDO HUMANISTA DE GUATEMALA': 'PHG',
    'PARTIDO HUMANISTA DE GUATEMALA  (PHG)': 'PHG',
    'PARTIDO HUMANISTA DE GUATEMALA (PHG': 'PHG',
    'PARTIDO HUMANISTA DE GUATEMALA (PHG)': 'PHG',
    'PARTIDO HUMANISTA DE GUATEMALA(PHG)': 'PHG',
    'PARTIDO HUMANISTA DE GUATEMALA(PHG) ': 'PHG',
    'PARTIDO HUMANISTA DE GUTEMALA (PHG)': 'PHG',
    'PARTIDO HUMANISTA GUATEMALA': 'PHG',
    'PARTIDO HUMANISTA GUATEMALA (PHG)': 'PHG',
    'Partido Humanista': 'PHG',
    'Partido Humanista de Guatemala (PHG)': 'PHG',
    'humanista': 'PHG',
    'humanista ': 'PHG',
    'partido humanista': 'PHG',
    'partido humanista de guatemala': 'PHG',
    'PARTIDO HUMASITA DE GUATEMALA (PHG) ': 'PHG',
    'PHG': 'PHG',
    'PHG ': 'PHG',
    'PNG': 'PHG',

    'FAMILIA': 'MI FAMILIA',
    'MI FAMILIA': 'MI FAMILIA',
    'MI FAMILIA ': 'MI FAMILIA',
    'MI FAMILIA  (MI FAMILIA)': 'MI FAMILIA',
    'MI FAMILIA ( MI FAMILIA)': 'MI FAMILIA',
    'MI FAMILIA (MI FAMILIA )': 'MI FAMILIA',
    'MI FAMILIA (MI FAMILIA)': 'MI FAMILIA',
    'MI FAMILIA (MI FAMILIA) ': 'MI FAMILIA',
    'MI FAMILIA(MI FAMILIA)': 'MI FAMILIA',
    'Mi Familia': 'MI FAMILIA',
    'Mi familia': 'MI FAMILIA',
    'Mi familia ': 'MI FAMILIA',
    'MiFamilia': 'MI FAMILIA',
    'familia': 'MI FAMILIA',
    'mi familia': 'MI FAMILIA',
    'mi famili': 'MI FAMILIA',
    'FAMILIA': 'MI FAMILIA',

    'INTEGRAL NACIONAL': 'PIN',
    'PARTIDO DE INTEGRACION NACIONAL': 'PIN',
    'PARTIDO DE INTEGRACION NACIONAL ': 'PIN',
    'PARTIDO DE INTEGRACION NACIONAL (PIM)': 'PIN',
    'PARTIDO DE INTEGRACION NACIONAL (PIN)': 'PIN',
    'PARTIDO DE INTEGRACION NACIONAL (PIR)': 'PIN',
    'PARTIDO DE INTEGRACION NACIONAL(PIN)': 'PIN',
    'PARTIDO DE INTEGRACIÒN NACIONAL (PIN)': 'PIN',
    'PARTIDO DE INTEGRACIÓN NACIONAL': 'PIN',
    'PARTIDO DE INTEGRACIÓN NACIONAL (PIN)': 'PIN',
    'PARTIDO DE INTEGRACIÓN NACIONAL(PIN) ': 'PIN',
    'PARTIDO INTEGRACION NACIONAL': 'PIN',
    'PARTIDO INTEGRACION NACIONAL (PIN)': 'PIN',
    'PARTIDO INTEGRACIÓN NACIONAL': 'PIN',
    'PATIDOS INTEGRACION NACIONAL (PIN)': 'PIN',
    'Partido de Integración Nacional': 'PIN',
    'Partido de Integración Nacional (PIN)': 'PIN',
    'Partido de Integración Nacional PIN': 'PIN',
    'partido de integracion nacional': 'PIN',
    'PIN': 'PIN',
    'PIN ': 'PIN',
    'Pin': 'PIN',
    'pin': 'PIN',
    'P.I.N': 'PIN',

    'M L P': 'MLP',
    'MLP': 'MLP',
    'MOVIEMIENTO PARA LA LIBERACION DE LOS PUEBLOS (MLP)': 'MLP',
    'MOVIMENTO PARA LA LIBERACION DE LOS PUEBLOS (MLP)': 'MLP',
    'MOVIMIENTO DE LA LIBERACION DE LOS PUEBLOS (MLP)': 'MLP',
    'MOVIMIENTO PARA LA  LIBERACION DE LOS PUEBLOS (MLP) ': 'MLP',
    'MOVIMIENTO PARA LA LIBERACION DE LOS PUEBLOS  (MLP)': 'MLP',
    'MOVIMIENTO PARA LA LIBERACION DE LOS PUEBLOS (MLP)': 'MLP',
    'MOVIMIENTO PARA LA LIBERACION DE LOS PUEBLOS (MLP) ': 'MLP',
    'MOVIMIENTO PARA LA LIBERACION DEL LOS PUEBLOS (MLP)': 'MLP',
    'MOVIMIENTO PARA LA LIBERACION POR LOS PUEBLOS (MLP)': 'MLP',
    'MOVIMIENTO PARA LA LIBERACIÓN DE LOS PUEBLOS (MLP)': 'MLP',
    'MOVIMIENTO PARA LA LIBERTAD DE LOS PUEBLOS (MLP)': 'MLP',
    'MOVIMIENTO PARA LA LINERACIÓN DE LOS PUEBLOS (MLP)': 'MLP',
    'MOVIMIENTO PARA LA LIVERACION DE LOS PUEBLOS (MLP) ': 'MLP',
    'MlP': 'MLP',
    'Movimiento para la Liberación de los Pueblos (MLP)': 'MLP',
    'mlp': 'MLP',
    'MOVIMIENTO PARA LA LIBERACION DE LOS PUEBLOS': 'MLP',
'movimiento para la liberación de los pueblos ': 'MLP',

    #// URNG-WINAQ
    'MAIZ URNG-WINAQ': 'URNG-MAIZ WINAQ',
    'URNG -MAIZ WINAQ (URNG- MAIZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG MAIZ WINAQ': 'URNG-MAIZ WINAQ',
    'URNG MAIZ WINAQ (URNG MAIZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG MAIZ WINAQ (URNG-MAIZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG MAIZ WINAQ( URGN MAIZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG MAIZ WINAQ(URNG-MAIZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG MAIZ- WINAQ': 'URNG-MAIZ WINAQ',
    'URNG MAIZ-WINAQ': 'URNG-MAIZ WINAQ',
    'URNG MAÍZ WINAQ(URNG MAÍZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG MAÍZ WINAQ(URNG-MAIZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG WINAQ': 'URNG-MAIZ WINAQ',
    'URNG, MAIZ WINAQ': 'URNG-MAIZ WINAQ',
    'URNG, Winaq': 'URNG-MAIZ WINAQ',
    'URNG- MAIZ WINAQ (URNG- MAIZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG-MAIZ -WINAQ': 'URNG-MAIZ WINAQ',
    'URNG-MAIZ WINAQ': 'URNG-MAIZ WINAQ',
    'URNG-MAIZ WINAQ ': 'URNG-MAIZ WINAQ',
    'URNG-MAIZ WINAQ  (URNG-MAIZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG-MAIZ WINAQ (URN-MAIZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG-MAIZ WINAQ (URNG-MAIZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG-MAIZ WINAQ (URNG-MAÍZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG-MAIZ WINAQ 8URNG-MAIZ WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG-MAIZ-WINAQ': 'URNG-MAIZ WINAQ',
    'URNG-MAIZ-WINAQ (URNG-MAIZ-WINAQ)': 'URNG-MAIZ WINAQ',
    'URNG-WINAQ': 'URNG-MAIZ WINAQ',
    'WINAQ Y URNG': 'URNG-MAIZ WINAQ',
    'UNRG - MAIZ - WINAQ': 'URNG-MAIZ WINAQ',
    'URG Maíz WINAQ ': 'URNG-MAIZ WINAQ',
    'URGN MAIZ WINAQ': 'URNG-MAIZ WINAQ',
    'URGN WINAQ': 'URNG-MAIZ WINAQ',
    'URNG MAIZ GUINAQ': 'URNG-MAIZ WINAQ',
    'URNG MAIZ WINAK': 'URNG-MAIZ WINAQ',

    #// WINAQ solo
    'WINAK': 'WINAQ',
    'WINAQ': 'WINAQ',
    'WINAQ ': 'WINAQ',
    'Winaq': 'WINAQ',
    'MOVIMIENTO POLITICO WINAQ (WINAQ)': 'WINAQ',
    'MOVIMIENTO POPULAR WINAQ (WINAQ)': 'WINAQ',
    
    #// URNG solo
    'UNIDAD REVOLUCIONARIA NACIONAL GUATEMALTECA (U.R.N.G.MAIZ ': 'U R N G - MAIZ',
    'UNIDAD REVOLUCIONARIA NACIONAL GUATEMALTECA (URNG - MAIZ)': 'U R N G - MAIZ',
    'UNIDAD REVOLUCIONARIA NACIONAL GUATEMALTECA (URNG MAIZ)': 'U R N G - MAIZ',
    'UNIDAD REVOLUCIONARIA NACIONAL GUATEMALTECA (URNG-MAIZ)': 'U R N G - MAIZ',
    'UNIDAD REVOLUCIONARIA NACIONAL GUETMALTECA (U R N G - MAIZ)': 'U R N G - MAIZ',
    'URGN MAIZ': 'U R N G - MAIZ',
    'URNG': 'U R N G - MAIZ',
    'URNG ': 'U R N G - MAIZ',
    'URNG MAIZ': 'U R N G - MAIZ',
    'URNG maiz': 'U R N G - MAIZ',
    'URNG-MAIZ': 'U R N G - MAIZ',
    'URNG-MAIZ ': 'U R N G - MAIZ',
    'urn maiz': 'U R N G - MAIZ',
    'urng': 'U R N G - MAIZ',
    'urng maiz': 'U R N G - MAIZ',


    #// URNG-WINAQ SEMILLA
    'URNG, WINAQ, SEMILLA': 'FOPPA X LA CIUDAD',
    'URNG-MAIZ SEMILLA (URNG-MAIZ SEMILLA)': 'FOPPA X LA CIUDAD',

    #// SEMILLA
    'MOVIMIENTO SEMILLA': 'SEMILLA',
    'MOVIMIENTO SEMILLA ': 'SEMILLA',
    'MOVIMIENTO SEMILLA (SEMILLA)': 'SEMILLA',
    'MOVIMIENTO SEMILLA (SEMILLA) ': 'SEMILLA',
    'MOVIMIENTO SEMILLA(SEMILLA)': 'SEMILLA',
    'Movimiento Semilla': 'SEMILLA',
    'SEMILLA': 'SEMILLA',
    'SEMILLA ': 'SEMILLA',
    'Semilla': 'SEMILLA',
    'movimiento semilla': 'SEMILLA',
    'semilla': 'SEMILLA',
    'SE,MILLA': 'SEMILLA',
    'SEMlLLA': 'SEMILLA',

    'MOSOTROS': 'PPN',
    'NOSOTROS': 'PPN',
    'NOSOTROS ': 'PPN',
    'Nosotros': 'PPN',
    'PARTIDO NOSOTROS': 'PPN',
    'PARTIDO PIOLITICO NOSOTROS (PPN)': 'PPN',
    'PARTIDO POLITICO NOSOTROS': 'PPN',
    'PARTIDO POLITICO NOSOTROS ': 'PPN',
    'PARTIDO POLITICO NOSOTROS (NOSOTROS)': 'PPN',
    'PARTIDO POLITICO NOSOTROS (PPN)': 'PPN',
    'PARTIDO POLITICO NOSOTROS (PPN) ': 'PPN',
    'PARTIDO POLITICO NOSOTROS(PPN)': 'PPN',
    'PARTIDO POLÍTICO NOSOTROS (PPN)': 'PPN',
    'PARTIDO POLÍTICO NOSOTROS(PPN)': 'PPN',
    'PARTIDOS POLITICOS NOSOTROS (PPN)': 'PPN',
    'PPARTIDO POLITICO NOSOTROS ': 'PPN',
    'Partido Político Nosotros': 'PPN',
    'Partido Político Nosotros (PPN)': 'PPN',
    'Partído Político Nosotros': 'PPN',
    'nosotros': 'PPN',
    'partido politico nosotros': 'PPN',
    'partido político nosotros ': 'PPN',
    'PPN': 'PPN',
    'PPN ': 'PPN',
    'NOSOTRS': 'PPN',
    'NOSTROS': 'PPN',

    'P A N': 'P A N',
    'P A N ': 'P A N',
    'PAN': 'P A N',
    'PAN ': 'P A N',
    'PARTIDA DE AVANZADA NACIONAL (PAN)': 'P A N',

    'PARTIDO AVANZA NACIONAL (PAN)': 'P A N',
    'PARTIDO AVANZADA NACIONAL (PAN)': 'P A N',
    'PARTIDO DE AVANSE NACUONAL (PAN)': 'P A N',
    'PARTIDO DE AVANZA NACIONAL (PAN)': 'P A N',
    'PARTIDO DE AVANZADA NACIONAL': 'P A N',
    'PARTIDO DE AVANZADA NACIONAL ( PAN)': 'P A N',
    'PARTIDO DE AVANZADA NACIONAL (P A N)': 'P A N',
    'PARTIDO DE AVANZADA NACIONAL (PAN)': 'P A N',
    'PARTIDO PAN': 'P A N',
    'Pan': 'P A N',
    'pan': 'P A N',
    'partido pan': 'P A N',

    'PARTIDO POPULAR GUATEMALTECO': 'PPG',
    'PARTIDO POPULAR GUATEMALTECO ': 'PPG',
    'PARTIDO POPULAR GUATEMALTECO (PPG)': 'PPG',
    'PARTIDO POPULAR GUATEMALTECO (PPG) ': 'PPG',
    'PARTIDO PUPULAR GUATEMALTECO': 'PPG',
    'PARTIDO PUPULAR GUATEMALTECO (PPG)': 'PPG',
    'PPG': 'PPG',
    'POPULAR GUATEMALTECO': 'PPG',
    'partido popular ': 'PPG',

    'PARTIDO  REPUBLICANO': 'PR',
    'PARTIDO REPUBLICA (PR)': 'PR',
    'PARTIDO REPUBLICANO': 'PR',
    'PARTIDO REPUBLICANO ': 'PR',
    'PARTIDO REPUBLICANO (PR)': 'PR',
    'PARTIDO REPUBLICANO (PR) ': 'PR',
    'PARTIDO REPUBLICANO(PR)': 'PR',
    'PARTIDOS REPUBLICANO (PR)': 'PR',
    'PATIDO REPUBLICANO (PR)': 'PR',
    'Partido Republicano': 'PR',
    'Partido Republicano (PR)': 'PR',
    'Partido Republicano PR': 'PR',
    'PARTIDO REPUNLICANO (PR)': 'PR',
    'PR': 'PR',
    'PR ': 'PR',
    'partido republicano': 'PR',

    'COMUNIDAD REPUBLICANA (UR)': 'UR',
    'REPUBLICANA GUATEMALTECA': 'UR',
    'UNIDAD REPUBLICANA ': 'UR',
    'UNION REPUBLICA': 'UR',
    'UNION REPUBLICA (UR)': 'UR',
    'UNION REPUBLICANA': 'UR',
    'UNION REPUBLICANA ': 'UR',
    'UNION REPUBLICANA (UR)': 'UR',
    'UNION REPUBLICANA (UR) ': 'UR',
    'UNION REPUBLICANA(UR)': 'UR',
    'UNION REPUBLICANO': 'UR',
    'UNION REPUBLICANO (UR)': 'UR',
    'UNIÓN REPUBLICANA': 'UR',
    'UNIÓN REPUBLICANA (UR)': 'UR',
    'Union Republicana': 'UR',
    'UnionRepublicana': 'UR',
    'Unión Republicana (UR)': 'UR',
    'union republicana': 'UR',
    'union republicana ': 'UR',
    'unión republicana': 'UR',
    'Unión Repúblicana UR': 'UR',
    'Ur': 'UR',
    'UNION REPLICANO (UR)': 'UR',
    'UNION REPUBICANA (UR)': 'UR',
    'UNIÓN REPÚBLICANA (UR)': 'UR',
    'UR': 'UR',
    'UR ': 'UR',
    'unión repúblicana': 'UR',

    'PARTIDO TODOS': 'TODOS',
    'TODOS': 'TODOS',
    'TODOS ': 'TODOS',
    'TODOS  (TODOS)': 'TODOS',
    'TODOS (TODOS )': 'TODOS',
    'TODOS (TODOS)': 'TODOS',
    'TODOS(TODOS)': 'TODOS',
    'Todos': 'TODOS',
    'Todos (Todos)': 'TODOS',
    'todos': 'TODOS',

    'PARTIDO UNIONISTA': 'UNIONISTA',
    'PARTIDO UNIONISTA (UNIONISTA)': 'UNIONISTA',
    'UNIONISTA': 'UNIONISTA',

    'VALOR  UNIONISTA': 'VALOR UNIONISTA',
    'VALOR (UNIONISTA)': 'VALOR UNIONISTA',
    'VALOR - UNIONISTA': 'VALOR UNIONISTA',
    'VALOR UNIONISTA': 'VALOR UNIONISTA',
    'VALOR UNIONISTA ': 'VALOR UNIONISTA',
    'VALOR UNIONISTA  (VALOR UNIONISTA)': 'VALOR UNIONISTA',
    'VALOR UNIONISTA (VALOR UNINISTA)': 'VALOR UNIONISTA',
    'VALOR UNIONISTA (VALOR UNIONISTA )': 'VALOR UNIONISTA',
    'VALOR UNIONISTA (VALOR UNIONISTA)': 'VALOR UNIONISTA',
    'VALOR UNIONISTA (VALOR UNIONISTA) ': 'VALOR UNIONISTA',
    'VALOR UNIONISTA(VALOR UNIONISTA)': 'VALOR UNIONISTA',
    'VALOR UNIONISTAS (VALOR UNIONISTAS)': 'VALOR UNIONISTA',
    'VALOR-UNIONISTA': 'VALOR UNIONISTA',
    'VALOR-UNIONISTA ': 'VALOR UNIONISTA',
    'VOLOR UNIONISTA ': 'VALOR UNIONISTA',
    'Valor Unionista': 'VALOR UNIONISTA',
    'Valor unionista': 'VALOR UNIONISTA',
    'valor unionista': 'VALOR UNIONISTA',
    'valor unionista ': 'VALOR UNIONISTA',
    'VALOR UNINISTA (VALOR UNINISTA )': 'VALOR UNIONISTA',
    'VALOR UNIONIONISTA': 'VALOR UNIONISTA',
    'VALOR UNISTA': 'VALOR UNIONISTA',

    'VALOR': 'VALOR',
    'VALOR ': 'VALOR',
    'VALOR (VALOR)': 'VALOR',
    'VALOR(VALOR)': 'VALOR',
    'Valor': 'VALOR',
    'valor': 'VALOR',
    'valor ': 'VALOR',

    'PODEMOS': 'PODEMOS',
    'PODEMOS ': 'PODEMOS',
    'PODEMOS (PODEMOS)': 'PODEMOS',
    'PODEMOS(PODEMOS)': 'PODEMOS',
    'PODEMOS(PODEMOS) ': 'PODEMOS',
    'Podemos': 'PODEMOS',
    'Podemos (PODEMOS)': 'PODEMOS',
    'podemos': 'PODEMOS',
    'PODENOS': 'PODEMOS',

    'TOTAL': 'TOTAL',
    'TOTAL ': 'TOTAL',
    'TOTAL DE VOT0S': 'TOTAL',
    'TOTAL DE VOTOS': 'TOTAL',
    'TOTAL DE VOTOS ': 'TOTAL',
    'TOTAL DE VOTOS   ': 'TOTAL',
    'TOTAL VOTOS': 'TOTAL',
    'TOTAL VOTOS ': 'TOTAL',
    'TOTAL VOTOS   ': 'TOTAL',
    'Total': 'TOTAL',
    'Total ': 'TOTAL',
    'total': 'TOTAL',
    'VOTOS TOTAL': 'TOTAL',


    'PODER': 'PODER',
    'Poder': 'PODER',
    'PARTIDO DE OPORTUNIDAD Y DESARROLLO (PODER)': 'PODER',

    #// comites
    'COCIG': 'COCIG',
    'COMITE CHERIFON': 'COMITE CHERIFON',
    'COMITE CIVICO JUSTICIA': 'COMITE CIVICO JUSTICIA',
    'COMITE CIVICO TRABAJANDO': 'COMITE CIVICO TRABAJANDO',
    'COMITE SIVICO DESARROLLO SHECANO': 'COMITE SIVICO DESARROLLO SHECANO',

    'CONSOL': 'CONSOL',
    'CamiteCivicoLaDeAgua': 'CamiteCivicoLaDeAgua',
    'FUTURO': 'FUTURO',
    'Gallo': 'Gallo',
    'Garrido': 'Garrido',
    
    
    'SBX': 'SBX',
    'CONVERGENCIA': 'CONVERGENCIA',
    'MALP': 'MALP',
    'MAPA': 'MAPA',
    'Movimiento plitico': 'Movimiento plitico',
    'REPUBLICANO': 'REPUBLICANO',
    'PRG': 'PRG',
    'PX': 'PX',
    'republicano': 'republicano',
}